In [1]:
import duckdb, os, pandas as pd

CSV_PATH = "/Users/joaodossantos/data/nyc_taxi/yellow_tripdata_2015-01.csv"

con = duckdb.connect()
print("DuckDB version:", duckdb.__version__)
print("CSV exists locally?", os.path.exists(CSV_PATH))

DuckDB version: 1.4.0
CSV exists locally? True


In [2]:
con.execute(f"""
CREATE OR REPLACE VIEW trips AS
SELECT
  CAST(VendorID AS INTEGER)                        AS vendor_id,
  CAST(tpep_pickup_datetime AS TIMESTAMP)          AS pickup_ts,
  CAST(tpep_dropoff_datetime AS TIMESTAMP)         AS dropoff_ts,
  CAST(passenger_count AS INTEGER)                 AS passenger_count,
  CAST(trip_distance AS DOUBLE)                    AS trip_distance,
  CAST(pickup_longitude AS DOUBLE)                 AS pickup_longitude,
  CAST(pickup_latitude  AS DOUBLE)                 AS pickup_latitude,
  CAST(RateCodeID AS INTEGER)                      AS ratecode_id,
  store_and_fwd_flag                                AS store_and_fwd_flag,
  CAST(dropoff_longitude AS DOUBLE)                AS dropoff_longitude,
  CAST(dropoff_latitude  AS DOUBLE)                AS dropoff_latitude,
  CAST(payment_type AS INTEGER)                    AS payment_type,
  CAST(fare_amount AS DOUBLE)                      AS fare_amount,
  CAST(extra AS DOUBLE)                            AS extra,
  CAST(mta_tax AS DOUBLE)                          AS mta_tax,
  CAST(tip_amount AS DOUBLE)                       AS tip_amount,
  CAST(tolls_amount AS DOUBLE)                     AS tolls_amount,
  CAST(improvement_surcharge AS DOUBLE)            AS improvement_surcharge,
  CAST(total_amount AS DOUBLE)                     AS total_amount
FROM read_csv_auto('{CSV_PATH}', IGNORE_ERRORS=true);
""")
con.sql("SELECT * FROM trips LIMIT 5").df()

,vendor_id,pickup_ts,dropoff_ts,passenger_count,trip_distance,pickup_longitude,pickup_latitude,ratecode_id,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30


In [3]:
con.sql("""
SELECT COUNT(*) AS total_trips,
       MIN(pickup_ts) AS first_trip,
       MAX(dropoff_ts) AS last_trip
FROM trips
""").df()

,total_trips,first_trip,last_trip
0,12748986,2015-01-01,2016-02-02 16:30:52


In [4]:
con.sql("SELECT ROUND(AVG(trip_distance), 2) AS avg_trip_miles FROM trips;").df()

,avg_trip_miles
0,13.46


In [5]:
con.sql("""
SELECT DATE_TRUNC('day', pickup_ts) AS day, COUNT(*) AS trips
FROM trips
WHERE pickup_ts >= TIMESTAMP '2015-01-01'
  AND pickup_ts <  TIMESTAMP '2015-02-01'
GROUP BY 1
ORDER BY trips DESC
LIMIT 10
""").df()

,day,trips
0,2015-01-31,520067
1,2015-01-10,515540
2,2015-01-30,483380
3,2015-01-16,478124
4,2015-01-17,476827
5,2015-01-23,472440
6,2015-01-24,460505
7,2015-01-22,452108
8,2015-01-15,451186
9,2015-01-08,450920


In [6]:
con.sql("""
SELECT DATE_TRUNC('day', pickup_ts) AS day,
       ROUND(SUM(total_amount), 2) AS revenue
FROM trips
WHERE pickup_ts >= TIMESTAMP '2015-01-01'
  AND pickup_ts <  TIMESTAMP '2015-02-01'
GROUP BY 1
ORDER BY day
""").df()

,day,revenue
0,2015-01-01,5843310.50
1,2015-01-02,5125959.57
2,2015-01-03,5738019.93
3,2015-01-04,5104766.03
4,2015-01-05,5526352.49
5,2015-01-06,5671972.45
6,2015-01-07,6178726.66
7,2015-01-08,6597447.71
8,2015-01-09,6709467.82
9,2015-01-10,7177624.24


In [7]:
con.sql("""
WITH t AS (
  SELECT payment_type, COUNT(*) AS trips
  FROM trips
  WHERE pickup_ts >= TIMESTAMP '2015-01-01'
    AND pickup_ts <  TIMESTAMP '2015-02-01'
  GROUP BY 1
)
SELECT payment_type, trips,
       ROUND(100.0 * trips / SUM(trips) OVER (), 2) AS pct
FROM t
ORDER BY trips DESC
""").df()

,payment_type,trips,pct
0,1,7881388,61.82
1,2,4816992,37.78
2,3,38632,0.30
3,4,11972,0.09
4,5,2,0.00


In [8]:
con.sql("""
SELECT EXTRACT(HOUR FROM pickup_ts) AS hour, COUNT(*) AS trips
FROM trips
WHERE pickup_ts >= TIMESTAMP '2015-01-01'
  AND pickup_ts <  TIMESTAMP '2015-02-01'
GROUP BY 1
ORDER BY 1
""").df()

,hour,trips
0,0,469971
1,1,355145
2,2,268133
3,3,198524
4,4,143271
5,5,127437
6,6,268455
7,7,456127
8,8,561802
9,9,580034


In [9]:
con.sql("""
SELECT
  quantile_cont(tip_amount, 0.25) AS p25,
  quantile_cont(tip_amount, 0.50) AS p50,
  quantile_cont(tip_amount, 0.75) AS p75
FROM trips
WHERE pickup_ts >= TIMESTAMP '2015-01-01'
  AND pickup_ts <  TIMESTAMP '2015-02-01'
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,p25,p50,p75
0,0.0,1.0,2.06


In [10]:
con.sql("""
WITH daily AS (
  SELECT DATE_TRUNC('day', pickup_ts) AS day,
         SUM(total_amount) AS revenue
  FROM trips
  WHERE pickup_ts >= TIMESTAMP '2015-01-01'
    AND pickup_ts <  TIMESTAMP '2015-02-01'
  GROUP BY 1
)
SELECT day,
       revenue,
       RANK() OVER (ORDER BY revenue DESC) AS revenue_rank,
       AVG(revenue) OVER (ORDER BY day ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS moving_avg_7d
FROM daily
ORDER BY day
""").df()

,day,revenue,revenue_rank,moving_avg_7d
0,2015-01-01,5.843311e+06,22,5.843311e+06
1,2015-01-02,5.125960e+06,28,5.484635e+06
2,2015-01-03,5.738020e+06,24,5.569097e+06
3,2015-01-04,5.104766e+06,29,5.453014e+06
4,2015-01-05,5.526352e+06,26,5.467682e+06
5,2015-01-06,5.671972e+06,25,5.501730e+06
6,2015-01-07,6.178727e+06,18,5.598444e+06
7,2015-01-08,6.597448e+06,14,5.706178e+06
8,2015-01-09,6.709468e+06,10,5.932393e+06
9,2015-01-10,7.177624e+06,6,6.138051e+06


In [11]:
import os
os.makedirs("artifacts", exist_ok=True)

daily_rev = con.sql("""
SELECT DATE_TRUNC('day', pickup_ts) AS day,
       ROUND(SUM(total_amount), 2) AS revenue
FROM trips
WHERE pickup_ts >= TIMESTAMP '2015-01-01'
  AND pickup_ts <  TIMESTAMP '2015-02-01'
GROUP BY 1
ORDER BY day
""").df()

daily_rev.to_csv("artifacts/daily_revenue_jan2015.csv", index=False)
daily_rev.head()

,day,revenue
0,2015-01-01,5843310.50
1,2015-01-02,5125959.57
2,2015-01-03,5738019.93
3,2015-01-04,5104766.03
4,2015-01-05,5526352.49
